# Advent of Code - 2024 - Day 20 - Problem 1

https://adventofcode.com/2024/day/20

## Load Source Data

Load the map data into `DATA`.

In [ ]:
f = open("data/day20.txt", "r")
DATA = list(map(str.strip, f.readlines()))
f.close()

## Create Map Class

In [2]:
class Map:

    def __init__(self, lines):
        self._lines = list(map(list, DATA))
        self._rows = len(self._lines)
        self._cols = len(self._lines[0])

        # Find the starting position
        #
        for row in range(self._rows):
            for col in range(self._cols):
                if self._lines[row][col] == "S":
                    self._start = (row, col)
                if self._lines[row][col] == "E":
                    self._end = (row, col)

    def can_move(self, target):
        return target != "#"

    def get_next_positions(self, position):
        row, col = position
        if row > 0:
            target = self._lines[row - 1][col]
            if self.can_move(target):
                yield (row - 1, col)
        if row < self._rows - 1:
            target = self._lines[row + 1][col]
            if self.can_move(target):
                yield (row + 1, col)
        if col > 0:
            target = self._lines[row][col - 1]
            if self.can_move(target):
                yield (row, col - 1)
        if col < self._cols - 1:
            target = self._lines[row][col + 1]
            if self.can_move(target):
                yield (row, col + 1)

    def find_paths(self, path):

        last_position = path[-1]
        for next_position in self.get_next_positions(last_position):
            if not next_position in path:
                new_path = list(path)
                new_path.append(next_position)
                if next_position == self._end:
                    yield new_path
                else:
                    yield from self.find_paths(new_path)

    def find_cheats(self, location, length):

        cheats = set()

        location_row, location_col = location

        for row in range(length + 1):
            for col in range(length - row + 1):
                if location_row + row < self._rows:
                    if location_col + col < self._cols:
                        cheats.add(((location_row + row, location_col + col), row + col))
                    if location_col + col > 0:
                        cheats.add(((location_row + row, location_col - col), row + col))
                if location_row - row > 0:
                    if location_col + col < self._cols:
                        cheats.add(((location_row - row, location_col + col), row + col))
                    if location_col + col > 0:
                        cheats.add(((location_row - row, location_col - col), row + col))

        return cheats

## Find Solutions

In [3]:
import sys

sys.setrecursionlimit(10000)

m = Map(DATA)

starting_path = [m._start]

# Find the path without cheating
#
for path in m.find_paths(starting_path):
    honest_path = path
    break

honest_path_length = len(honest_path)

lengths_to_exit = dict()
for idx in range(len(honest_path)):
    location = honest_path[idx]
    length_to_exit = honest_path_length - idx
    lengths_to_exit[location] = length_to_exit

improvements = dict()
for idx in range(len(honest_path)):
    position = honest_path[idx]
    for cheat_position, cheat_length in m.find_cheats(position, 2):
        if cheat_position in lengths_to_exit:
            cheat_length = idx + cheat_length + lengths_to_exit[cheat_position]
            improvement = honest_path_length - cheat_length
            if improvement >= 100:
                if improvement not in improvements:
                    improvements[improvement] = 1
                else:
                    improvements[improvement] += 1

total_count = 0
for improvement in improvements:
    total_count += improvements[improvement]
print(f"total_count = {total_count}")

total_count = 1521
